In [ ]:
# Render our plots inline
%matplotlib inline

from requests import get
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from pandas.io.json import json_normalize

Basic settings

In [ ]:
url = 'http://localhost:8080/'

Get the model contents: concept, casebase, attribute, attribute types

In [ ]:
def get_casebases():
    raw = pd.DataFrame(requests.get(url + 'casebase/').json()) 
    casebases = pd.DataFrame.from_records(raw).values.tolist()
    return casebases

def get_concepts():
    raw = pd.DataFrame(requests.get(url + 'concepts/').json()) 
    concepts = pd.DataFrame.from_records(raw).values.tolist()
    return concepts

def get_globalsim(concept):
    raw = pd.DataFrame(requests.get(url + 'concepts/' + concept + '/amalgamationFunctions').json()) 
    global_sim = pd.DataFrame.from_records(raw).values.tolist()
    return global_sim

def get_attributes(concept):
    raw = pd.DataFrame(requests.get(url + 'concepts/' + concept + '/attributes').json()) 
    attributes = pd.DataFrame(raw).reset_index()
    attributes.columns = ['name', 'type']
    return attributes

def get_instances(concept, casebase):
    raw = pd.DataFrame(requests.get(url + 'concepts/' + concept + '/casebases/' + casebase + '/instances').json()) 
    instances = raw.apply(pd.to_numeric, errors='coerce').fillna(raw)
    return instances

In [ ]:
def plot_numeric(data, attribute):
    ax = sns.distplot(data[attribute].astype(int), rug=True, hist=True, kde=False)
    ax.set(xlabel=attribute, ylabel='Count', title=attribute + ' (mean: '+data[attribute].astype(int).mean().__str__()+')')

def plot_symbolic(data, attribute):
    ax = sns.countplot(x=attribute, data=data)
    ax.set(xlabel=attribute, ylabel='Dist', title=attribute)

In [ ]:
get_concepts()
get_casebases()
get_instances('Car','CaseBase0').head()
#get_globalsim('Car')
#get_attributes('Car')

In [ ]:
plot_symbolic(get_instances('Car','CaseBase0'), 'Color')

In [ ]:
plot_numeric(get_instances('Car','CaseBase0'), 'Miles')

Retrieval

In [ ]:
def retrieve_k_sim(concept, casebase, queryID, k):
    raw = pd.DataFrame(requests.get(url + 'concepts/' + concept + '/casebases/' + casebase + '/retrievalByID?caseID=' + queryID + '&k=' + k ).json()) 
    results = raw.apply(pd.to_numeric, errors='coerce').fillna(raw).sort_values(by='similarCases', ascending=False)
    return results

retrieve_k_sim('Car', 'CaseBase0', '441_audi', '5')